In [1]:
import pandas as pd
import numpy as np

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor

from matplotlib import pyplot as plt

from sklearn import linear_model
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from itertools import combinations

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.feature_selection import SelectKBest, f_regression,mutual_info_regression
from sklearn.feature_selection import RFECV
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

import seaborn as sns
import pickle

In [2]:
with open('dropped_news_df_model', 'rb') as handle:
    dropped_news = pickle.load(handle)

In [42]:
dropped_news.head()

,n_tokens_title,n_tokens_content,n_non_stop_words,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_entertainment,...,avg_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares,day_of_week,LDA,lda,channels
0,12.0,219.0,1.0,4.0,2.000000,1.0,0.000001,4.680365,5.0,1.000000,...,-0.350000,0.500000,-0.187500,0.000001,0.187500,593,Monday,nan,nan,Entertainment
1,9.0,255.0,1.0,3.0,1.000000,1.0,0.000001,4.913725,4.0,0.000001,...,-0.118750,0.000001,0.000001,0.500000,0.000001,711,Monday,nan,nan,Business
2,9.0,211.0,1.0,3.0,1.000000,1.0,0.000001,4.393365,6.0,0.000001,...,-0.466667,0.000001,0.000001,0.500000,0.000001,1500,Monday,nan,nan,Business
3,9.0,531.0,1.0,9.0,0.000001,1.0,0.000001,4.404896,7.0,1.000000,...,-0.369697,0.000001,0.000001,0.500000,0.000001,1200,Monday,nan,nan,Entertainment
4,13.0,1072.0,1.0,19.0,19.000000,20.0,0.000001,4.682836,7.0,0.000001,...,-0.220192,0.454545,0.136364,0.045455,0.136364,505,Monday,nan,nan,Tech


# Dropping Columns that I Made for EDA

In [3]:
dropped_news.drop(columns=['day_of_week', 'LDA', 'lda', 'channels'], inplace=True)

# Creating Target and Feature Variables

In [4]:
target = np.log(dropped_news['shares'])
features = dropped_news.drop(columns='shares')

# Making All 0 Values into 0.000000001 Values

In [5]:
dropped_news.replace(to_replace=0, value=0.000001, inplace=True)

# Setting Up Train/Test Split with Polynomial Features

In [6]:
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=23,test_size=0.2)  

In [7]:
poly= PolynomialFeatures(degree=2, include_bias=False) # if you only want interactions without any squared features add interaction_only=True
poly_train=poly.fit_transform(X_train)
poly_test= poly.transform(X_test) # Not sure if this should be fit_transform or just transform
poly_cols=poly.get_feature_names(X_train.columns)
poly_cols=[col.replace(' ','X') for col in poly_cols]
X_train_poly=pd.DataFrame(data=poly_train,columns=poly_cols)
X_test_poly=pd.DataFrame(data=poly_test,columns=poly_cols)

# Scaling Data using Standard Scaler

In [8]:
scaler = StandardScaler()
scaler.fit(X_train_poly)
X_train_poly = pd.DataFrame(data=scaler.transform(X_train_poly), columns=X_train_poly.columns)
X_test_poly =pd.DataFrame(data=scaler.transform(X_test_poly), columns=X_train_poly.columns)

# Scaling Data Using MinMax Scaler

In [18]:
scaler = MinMaxScaler()
scaler.fit(X_train_poly)
X_train_poly = pd.DataFrame(data=scaler.transform(X_train_poly), columns=X_train_poly.columns)
X_test_poly =pd.DataFrame(data=scaler.transform(X_test_poly), columns=X_train_poly.columns)

###### I seem to be getting similar results with both a MinMax Scaler and a Standard Scaler after creating polynomial features

# Standard Linear Regression

In [9]:
#instantiate a linear regression object
lm = LinearRegression(normalize=True)

#fit the linear regression to the data
lm = lm.fit(X_train_poly, y_train)

y_train_pred = (lm.predict(X_train_poly))

train_rmse = (np.sqrt(metrics.mean_squared_error((y_train), y_train_pred)))
train_rmse = np.exp(train_rmse)

print('Training Root Mean Squared Error:' , train_rmse)

y_test_pred = (lm.predict(X_test_poly))

test_rmse = np.sqrt(metrics.mean_squared_error((y_test), y_test_pred))
test_rmse = np.exp(test_rmse)
# r_square = metrics.r2_score(y_train_pred, y_pred)

print('Testing Root Mean Squared Error:' , test_rmse)


print('Training R^2: ', float(metrics.r2_score(y_train, y_train_pred)), 'Testing R^2: ', float(metrics.r2_score(y_test, y_test_pred)))


Training Root Mean Squared Error: 2.3380910489678985
Testing Root Mean Squared Error: 2.402830037486477
Training R^2:  0.15754910876183226 Testing R^2:  0.1036408432220336


In [ ]:
sns.residplot(y_train, y_train_pred,lowess=True, color="g")

In [14]:
train_rmse / dropped_news.shares.std()
# normalized training rmse for standard regression model

0.00020180367253950897

In [15]:
test_rmse/dropped_news.shares.std()
# normalized test rmse for standard regression model

0.00020739137864928998

# F-Test Linear Regression Model

In [ ]:
# selector = SelectKBest(f_regression, k=10)
# selector.fit(X_train_poly, y_train)
# selected_ftest = X_train_poly.columns[selector.get_support()]
# removed_ftest = X_train_poly.columns[~selector.get_support()]

In [ ]:
# #instantiate a linear regression object
# lm_ftest = LinearRegression()

# #fit the linear regression to the data
# lm_ftest = lm_ftest.fit(X_train_poly[selected_ftest], y_train)

# y_train_pred_ftest = (lm_ftest.predict((X_train_poly[selected_ftest])))

# train_rmse_ftest = (np.sqrt(metrics.mean_squared_error((y_train), y_train_pred_ftest)))
# train_rmse_ftest = np.exp(train_rmse_ftest)


# print('Training Root Mean Squared Error:' , train_rmse_ftest)

# y_pred_ftest = (lm_ftest.predict((X_test_poly[selected_ftest])))

# test_rmse_ftest = np.sqrt(metrics.mean_squared_error((y_test), y_pred_ftest))
# test_rmse_ftest = np.exp(test_rmse_ftest)

# print('Testing Root Mean Squared Error:' , test_rmse_ftest)


# print("vs. Testing: ", float(test_rmse),
#       "vs. Testing ftest: ", float(test_rmse_ftest))

###### the f-test model took too long to run (I let it sit for an hour and a half | also after changing the cv) without returning a result, so I decided not to use it. I included it to show that I attempted to use it

# Recursive Feature Elimination Regression Model

In [49]:
# ols = linear_model.LinearRegression()

In [55]:
# # Create recursive feature eliminator that scores features by mean squared errors
# selector = RFECV(estimator=ols, step=1, cv=10, scoring='neg_mean_squared_error')

# # Fit recursive feature eliminator 
# selector.fit(X_train_poly, y_train)
# selected_rfe = X_train.columns[selector.support_]
# removed_rfe = X_train.columns[~selector.support_]

KeyboardInterrupt: 

In [ ]:
# #instantiate a linear regression object
# lm_rfe = LinearRegression()

# #fit the linear regression to the data
# lm_rfe = lm_rfe.fit(X_train[selected_rfe], y_train)

# y_train_pred_rfe = np.exp(lm_rfe.predict(X_train[selected_rfe]))

# train_rmse_rfe = np.sqrt(metrics.mean_squared_error((y_train), y_train_pred_rfe))


# print('Training Root Mean Squared Error:' , train_rmse_rfe)

# y_pred_rfe = np.exp(lm_rfe.predict(X_test[selected_rfe]))

# test_rmse_rfe = np.sqrt(metrics.mean_squared_error((y_test), y_pred_rfe))

# print('Testing Root Mean Squared Error:' , test_rmse_rfe)


# print("vs. Testing: ", float(test_rmse), 
#       "vs. Testing rfe: ", float(test_rmse_rfe))

###### I ran into a similar issue with the RFE model as I did with the F-Test Model

# Lasso Linear Regression Model

In [10]:
lasso = Lasso(alpha=0.05, max_iter = 5000, normalize=False)

final_lasso = lasso.fit(X_train_poly,y_train)

y_train_pred_las = np.exp(lasso.predict(X_train_poly))
y_test_pred_las = np.exp(lasso.predict(X_test_poly))

train_rmse_las = (metrics.mean_absolute_error(y_train, y_train_pred_las))
# train_rmse_las = np.exp(train_rmse_las)

test_rmse_las = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred_las))
# test_rmse_las = np.exp(test_rmse_las)


print('Training Error: '+ str(train_rmse_las))
print('Testing Error: '+ str(test_rmse_las))
print('Training R^2: ', float(metrics.r2_score(y_train, np.log(y_train_pred_las))), 'Testing R^2: ', float(metrics.r2_score(y_test, np.log(y_test_pred_las))))


Training Error: 1773.6016889271045
Testing Error: 1805.9040898910057
Training R^2:  0.0726976829820336 Testing R^2:  0.07024109951847812


In [ ]:
sns.residplot(y_train, y_train_pred_las,lowess=True, color="g")

In [13]:
train_rmse_las/dropped_news.shares.std()
# normalized training rmse for lasso regression model

0.15308186334563895

In [16]:
test_rmse_las/dropped_news.shares.std()
# normalized test rmse for lasso regression model

0.15586992549113862

# Ridge Linear Regression Model

In [11]:
ridge = Ridge(alpha=0.05, max_iter = 5000, normalize=False)

final_ridge = ridge.fit(X_train_poly, y_train)

y_train_pred_rid = ridge.predict(X_train_poly)
y_test_pred_rid = ridge.predict(X_test_poly)

train_rmse_rid = (np.sqrt(metrics.mean_squared_error(y_train,y_train_pred_rid)))
test_rmse_rid = (np.sqrt(metrics.mean_squared_error(y_test, y_test_pred_rid)))

print('Training Error: '+ str(train_rmse_rid))
print('Testing Error: '+ str(test_rmse_rid))
print('Training R^2: ', float(metrics.r2_score(y_train, y_train_pred_rid)), 'Testing R^2: ', float(metrics.r2_score(y_test, y_test_pred_rid)))


Training Error: 0.8518884132867331
Testing Error: 0.878059514893663
Training R^2:  0.15247567184666022 Testing R^2:  0.10075042460807959


In [ ]:
sns.residplot(y_train, y_train_pred_rid,lowess=True, color="g")

In [17]:
train_rmse_rid/dropped_news.shares.std()

7.352759443264868e-05

In [18]:
test_rmse_rid/dropped_news.shares.std()

7.578645617415976e-05